# Decentralized Multi-Agent Pathfinding: A Game-Theoretic Approach

### Introduction

In contrast to **centralized** planners which possess a global "god's-eye view" to compute optimal plans for all agents simultaneously, **decentralized** approaches grant autonomy to individual agents. Each agent plans for itself and coordinates with others to resolve conflicts. This paradigm models agents as rational actors who are myopic (planning for themselves) but socially compliant (obeying a set of coordination rules).

We can frame this coordination challenge as a cooperative game. The algorithm we will model is **Prioritized Planning**, where agents follow a strict social hierarchy to negotiate who gets the right-of-way.

***

### The Base Game: Prioritized Planning in an Open Environment

We first consider the base case where the only obstacles are the other agents.

#### Game Formulation (Base Case)

This is a **fully cooperative game** where all players win or lose as a team.

* **Players**: A set of $k$ agents, $\mathcal{A} = \{A_1, A_2, \dots, A_k\}$.
* **Environment**: A graph $G=(V, E)$ representing the grid.
* **Objective**: To find a set of conflict-free paths, $\Pi = \{\pi_1, \pi_2, \dots, \pi_k\}$, that brings every agent from its start location $s_i$ to its goal location $g_i$. The team's score is the **sum-of-costs**, which we want to minimize.
    \begin{aligned}
    \text{Cost}(\Pi) = \sum_{i=1}^{k} |\pi_i|
    \end{aligned}
    where $|\pi_i|$ is the length (cost) of the path for agent $A_i$.

* **Agent Capability**: Each agent $A_i$ is equipped with a single-agent pathfinding algorithm (e.g., A*) capable of finding an optimal path for itself given a set of dynamic space-time constraints $\mathcal{C}$.

#### The Rules of Play (The Algorithm)

The game proceeds sequentially according to a pre-defined "social rule" of priority.

1.  **Establish Priority**: A strict, total ordering is defined over the agents before planning begins. This priority ranking is fixed.
    \begin{aligned}
    A_1 > A_2 > \dots > A_k
    \end{aligned}

2.  **Sequential Pathfinding**: The agents plan one-by-one, from highest to lowest priority.
    -   The highest-priority agent, $A_1$, plans its path $\pi_1$ in a completely empty environment. Its dynamic constraint set, $\mathcal{C}_0$, is empty.
        \begin{aligned}
        \pi_1 = \arg\min_{\pi} \text{cost}(\pi \mid \mathcal{C}_0 = \emptyset)
        \end{aligned}
    -   Every subsequent agent $A_i$ must compute its shortest path while treating the paths of all previously planned agents $\{A_1, \dots, A_{i-1}\}$ as dynamic obstacles defined in the constraint set $\mathcal{C}_{i-1}$.
        \begin{aligned}
        \pi_i = \arg\min_{\pi} \text{cost}(\pi \mid \mathcal{C}_{i-1})
        \end{aligned}
    -   After finding its path $\pi_i$, the locations it occupies are added to the constraint set for all agents that follow. The new constraint set $\mathcal{C}_i$ becomes:
        \begin{aligned}
        \mathcal{C}_i = \mathcal{C}_{i-1} \cup \{ (\pi_i(t), t) \mid t \in [0, \infty) \}
        \end{aligned}

#### Analysis of the Base Game
This decentralized "game" is a powerful heuristic, but it comes with significant trade-offs:
* **Optimality**: It is **not optimal**. A high-priority agent's selfish choice may lead to a poor overall team score.
* **Completeness**: It is **not complete**. A high-priority agent can block a low-priority agent in a way that makes a solution impossible to find, even if one exists.
* **Complexity**: Its primary advantage is speed. The time complexity is polynomial in the number of agents, roughly $k \times \text{Cost(A*)}$, which is highly scalable.

***

### Extension: Incorporating Static Obstacles

Now, let's see how the theory adapts when we add a permanent, static obstacle to the environment.

#### The Conceptual Change

Adding a static obstacle does not change the high-level, turn-based logic of Prioritized Planning. The agents still plan sequentially according to their priority. The change occurs at the **low level**: the single-agent pathfinding problem becomes more difficult.

Each agent's individual A* search must now avoid two types of constraints simultaneously:
1.  The **dynamic constraints** $\mathcal{C}$ from higher-priority agents' paths.
2.  A new set of **static constraints** $\mathcal{O}$ representing the obstacle.

#### Formalizing the Change

We introduce a set $\mathcal{O} \subset V$ containing all locations `(row, col)` that are permanently blocked by the obstacle.

The pathfinding problem for each agent $A_i$ is modified to include this new constraint set. The agent must find the path with the minimum cost that respects *both* sets.

The updated objective for agent $A_i$ is:
\begin{aligned}
\pi_i = \arg\min_{\pi} \text{cost}(\pi \mid \mathcal{C}_{i-1}, \mathcal{O})
\end{aligned}

This means that during its A* search, an agent must prune any potential move that would land it in a location $v$ where $v \in \mathcal{O}$.

#### Impact on Performance: Analyzing Your Benchmark Results

Your benchmark data perfectly demonstrates the effect of adding the static obstacle set $\mathcal{O}$.

| Metric | No Obstacle | With Obstacle | Change |
| :--- | :--- | :--- | :--- |
| **Solution Cost** | 38 | 41 | +3 |
| **Execution Time**| 0.0044 s | 0.0113 s| +157% |
| **States Processed**| 40 | 159 | +298% |

**Interpretation:**

1.  **Why did Solution Cost increase?** The obstacle set $\mathcal{O}$ made the original, optimal paths invalid. It constrained the search space, forcing one or both agents to find a longer detour. This directly increased the total path cost from 38 to 41.

2.  **Why did Execution Time and States Processed increase?** The obstacle blocked the "greedy" or most direct route for the A\* planner. Instead of moving in a nearly straight line to the goal, the planner hit the obstacle and had to backtrack and explore many more alternative nodes to find a way around. This increase in exploration is precisely measured by the **298% jump in "States Processed,"** which in turn caused the **157% increase in execution time**. The algorithm had to do significantly more work to solve a harder problem.